# Installation

In [1]:
%%bash
pip install  --upgrade\
    'vllm>=0.8.3' \
    'transformers>=4.50.3' \
    pyzmq \
    unsloth \
    accelerate \
    bitsandbytes \
    openai \
    langchain-text-splitters \
    peft \
    FlagEmbedding \
    datasets \
    faiss-cpu \
    langchain-text-splitters \
    tavily-python \
    "flashinfer-python>=0.2.4"  --extra-index-url https://flashinfer.ai/whl/cu124/torch2.6/
git clone https://github.com/ggml-org/llama.cpp.git
cd llama.cpp/gguf-py/ && pip install --editable .
pip install jupyter-kernel-gateway ipykernel
pip install autogen
# pip install --upgrade --no-deps numpy==1.26.4 pandas==2.2.2

Looking in indexes: https://pypi.org/simple, https://flashinfer.ai/whl/cu124/torch2.6/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61

In [2]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
!huggingface-cli login --add-to-git-credential --token $HF_TOKEN
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

Token is valid (permission: write).
The token `WriteToken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


```bash
VLLM_BACKEND=FLASHINFER VLLM_USE_V1=1 VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 TOKENIZERS_PARALLELISM=true MAX_JOBS=2 vllm serve ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g --port 8877 --max-model-len 4096 --api-key token-abc123 --quantization compressed-tensors --max-num-seqs=1
```

# PEFT

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
from peft import LoraConfig, get_peft_model
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    #device_map="cuda",
    trust_remote_code=True
)

# check https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaR

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable()  # reduce number of stored activations

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params

In [ ]:
ori_p = print_trainable_parameters(model)

trainable params: 0 || all params: 3,212,749,824 || trainable : 0.0%


In [ ]:
#model.unload()

In [ ]:
config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CAUSAL LANGUAGE MODELS (like Bloom, LLaMA) or SEQ TO SEQ (like FLAN, T5)
)

model = get_peft_model(model, config)

In [ ]:
peft_p = print_trainable_parameters(model)

trainable params: 4,587,520 || all params: 3,217,337,344 || trainable : 0.1425874724810952%


In [ ]:
import transformers
from datasets import load_dataset
dataset = load_dataset("kimjaewon/baemin_sft_data")

README.md:   0%|          | 0.00/420 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/311k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1452 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'positive_document_list', 'negative_document_list', 'answer'],
        num_rows: 1452
    })
})

In [ ]:
peft_dataset = dataset.remove_columns(['positive_document_list', 'negative_document_list'])
peft_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1452
    })
})

In [ ]:
peft_dataset['train']['question'][:8]

['정산이 이뤄지는 시점은 언제인가요?',
 '주문 차단을 하고 싶다면 어떻게 해야 하나요?',
 '탈퇴 회원의 댓글 작성자 본인 여부를 확인할 수 있는 방법은 없나요?',
 '네이버 플레이스에 가게 정보를 제공하면 연동 여부 반영까지 얼마나 걸리나요?',
 '어떤 음식은 배달의민족을 통해 판매할 수 없나요?',
 '어떤 경우에 고객센터로 문의해야 하나요?',
 '울트라콜과 오픈리스트 상품의 배달팁은 어떻게 구분되나요?',
 '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요?']

In [ ]:
def merge_cols(example):
    example["prediction"] = example["question"] + " [배민 데이터 참조] " + example["answer"]
    return example

peft_dataset['train'] = peft_dataset['train'].map(merge_cols) # <-- 모든 문장에 대해 처리해 줍니다.
peft_dataset['train']["prediction"][5:7]

Map:   0%|          | 0/1452 [00:00<?, ? examples/s]

['어떤 경우에 고객센터로 문의해야 하나요? [배민 데이터 참조] 고객의 개인정보와 관련된 문의와 함께 배달의 불만사항, 결제문제 등 배달 서비스와 관련된 문제가 발생한 경우, 그리고 배달 관련 추가 연락이 필요한 경우에는 고객센터로 문의해주시면 최대한 도움을 드리겠습니다.',
 '울트라콜과 오픈리스트 상품의 배달팁은 어떻게 구분되나요? [배민 데이터 참조] 울트라콜과 오픈리스트 상품의 배달팁은 용도에 따라 기본 배달팁과 할증 배달팁으로 나누어집니다. 기본 배달팁은 주문금액에 따른 최대 3개의 배달팁 설정이 가능하며, 할증 배달팁은 지역, 시간대, 공휴일 등을 위해 별도로 설정이 가능합니다.']

In [ ]:
peft_dataset['train'][0]

{'question': '정산이 이뤄지는 시점은 언제인가요?',
 'answer': '매출이 발생된 기점(D일)으로부터 영업일 기준 D+3일 이후 정산되어 지급됩니다.',
 'prediction': '정산이 이뤄지는 시점은 언제인가요? [배민 데이터 참조] 매출이 발생된 기점(D일)으로부터 영업일 기준 D+3일 이후 정산되어 지급됩니다.'}

In [ ]:
peft_dataset = peft_dataset.map(
                        lambda x: tokenizer(x['prediction']),
                        batched=True
                     )

Map:   0%|          | 0/1452 [00:00<?, ? examples/s]

In [ ]:
peft_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 1452
    })
})

In [ ]:
peft_dataset['train'][7]

{'question': '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요?',
 'answer': '아니요, 배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 불가능합니다. 배달지역 조회만 가능하며, 배달지역 수정이 필요한 경우 고객센터를 통해 문의하셔야 합니다.',
 'prediction': '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요? [배민 데이터 참조] 아니요, 배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 불가능합니다. 배달지역 조회만 가능하며, 배달지역 수정이 필요한 경우 고객센터를 통해 문의하셔야 합니다.',
 'input_ids': [128000,
  103588,
  101607,
  108157,
  102913,
  119524,
  74769,
  101607,
  116512,
  101204,
  125935,
  18918,
  102681,
  24486,
  74769,
  104684,
  120257,
  89613,
  13094,
  125502,
  122665,
  30,
  510,
  103588,
  101607,
  55348,
  103718,
  93917,
  60,
  104231,
  36811,
  11,
  74769,
  101607,
  108157,
  102913,
  119524,
  74769,
  101607,
  116512,
  101204,
  125935,
  18918,
  102681,
  24486,
  74769,
  104684,
  120257,
  89613,
  13094,
  102786,
  116669,
  61938,
  13,
  74769,
  104684,
  120257,
  98267,
  73653,
  96451,
  108859,
  11,
  74769,
  104684,
  120257,
  89613,
  13094,
  126168,
  50152,
  116534,
  110816,
  18918

In [ ]:
train_args=transformers.TrainingArguments(
        per_device_train_batch_size=30,

        warmup_steps=100,
        max_steps=60,

        learning_rate=2e-4,

        # -- peft -- #
        gradient_accumulation_steps=6,
        fp16=True,
        # ---------- #

        logging_steps=20,
        output_dir='outputs'
    )

tokenizer.pad_token = tokenizer.eos_token
model=model.to("cuda")

trainer = transformers.Trainer(
    model=model,
    train_dataset=peft_dataset['train'],
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kevintb to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,3.305100
40,2.921800
60,2.327100


TrainOutput(global_step=60, training_loss=2.851325289408366, metrics={'train_runtime': 1443.1261, 'train_samples_per_second': 7.484, 'train_steps_per_second': 0.042, 'total_flos': 2.2523741514141696e+16, 'train_loss': 2.851325289408366, 'epoch': 6.73469387755102})

In [ ]:
model_path = 'llama_peft'  # it will be directory
trainer.model.save_pretrained(model_path)

In [ ]:
lora_config = LoraConfig.from_pretrained(model_path)
model = get_peft_model(model, lora_config)

**Run PEFT Model on vLLM**
---
```bash
vllm serve meta-llama/Llama-3.2-3B-Instruct --port 8877 --dtype float16 --max-model-len 2048 --api-key token-abc123 --enable-lora --lora-modules '{"name": "llama_peft", "path": "llama_peft/", "base_model_name": "meta-llama/Llama-3.2-3B-Instruct"}' --load-format=bitsandbytes --quantization=bitsandbytes --gpu-memory-utilization 0.8
```

In [3]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [4]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "meta-llama/Llama-3.2-3B-Instruct",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""
    print("========= Llama-3.2-3B-Instruct ===========")

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [5]:
def generate_peft_response(message_list):
    completion = client.chat.completions.create(
        model = "llama_peft",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""
    print("========= llama_peft ===========")

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [11]:
message_list = [
    {"role": "system", "content": "당신은 유저의 질문에 최대한 정확하고 풍부한 정보를 전달하는 assistant 이다. 답변은 항상 한국어로 공손하게 답변해줘."},
    {"role": "user", "content": "안심번호와 함께 주문한 음식이 취소된 경우 어떻게 되나요? [배민 데이터 참조]"}
]

generate_response(message_list)
generate_peft_response(message_list)

========= Llama-3.2-3B-Instruct ===========
안심번호와 함께 주문한 음식을 취소한 경우, 배민은 다음과 같은 절차를 거치게 됩니다.

1.  주문 취소 요청: 유저가 배민 앱을 통해 주문 취소 요청을 보내면, 배민은 유저의 주문에 대한 정보를 확인하고, 주문이 취소되면 유저에게 취소 hoàn금을 제공합니다.
2.  주문 취소 확인: 배민은 유저의 주문이 취소된 경우에 대해 확인합니다. 이때, 배민은 주문을 취소한 이유를 확인하여, 유저가 주문을 취소한 이유가 사용자 Agreement의 제시된 조건과 일치하는지 확인합니다.
3.  취소 hoàn금 제공: 배민은 주문을 취소한 후에,유저에게 수령할 수 있는 완금액을 계산합니다. 이 완금액은 주문 가격의 100%에서 50%로 제한되며, 이 제한은 사용자 Agreement에 정의된 제정도입니다.
4.  완금액 지급: 배민은 usuario가 주문을 취소한 후에, 유서의 완금액을 지급합니다. 이 완금액은 유서의 사용자 Agreement에 정의된 방법으로 지급되며, 이 방법은 사용자 Agreement에 정의된 방법으로 지급되며, 이 방법은 사용자 Agreement에 정의된 방법으로 지급됩니다.

다만, 안심번호와 함께 주문한 음식을 취소한 경우는 다음과 같은 조건이 있습니다.

*   안심번호와 함께 주문 한 음식이 만든일보다 ít 시간이 지나지 않은 경우
*   안심번호와 함께 주문 한 음식이 만든일보다 24시간이 지나지 않은 경우

이러한 조건을 충족하지 못하는 경우에는 주문을 취소하는 것이 불가하므로, 이러한 조건을 충족하지 못하는 경우에는 주문을 취소할 수 없습니다.

inference time: 6.65838 sec 


========= llama_peft ===========
안심번호와 함께 주문한 음식이 취소된 경우, 주문상태가 '취소'로 변경됩니다. 이때, 주문서류는 취소된 음식과 관련된 정보만 포함되어 있습니다. 또한, 안심번호가 사용된 주문은 7일간 재주문이 가능합니다.

"안심번호와 함께 주문한 음식이 취소된 경우, 주문상태가 '취소'로 변경됩니다. 이때, 주문서류는 취소된 음식과 관련된 정보만 포함되어 있습니다. 또한, 안심번호가 사용된 주문은 7일간 재주문이 가능합니다."

In [13]:
message_list = [
    {"role": "system", "content": "당신은 유저의 질문에 최대한 정확하고 풍부한 정보를 전달하는 assistant 이다. 답변은 항상 한국어로 공손하게 답변해줘."},
    {"role": "user", "content": "성수동에 날라다니는 홍길동의 자취를 찾아줘"}
]

generate_response(message_list)
generate_peft_response(message_list)

========= Llama-3.2-3B-Instruct ===========
성수동에 있는 홍길동의 자취를 찾아보겠습니다.

hong Gil-dong (홍길동)이라는 이름은 한국의 전통적인 소설인 "홍Gil-dong"으로부터 유래되었습니다. 이 소설은 18세기 중반에 써진 한국의 소설로, 홍길동은 한 명의 주인공으로서 사람들을 도와주고善을 일으키는 인물입니다.

hong Gil-dong이 성수동에 살았던 장소는 "성수동 Hong-gil-dong"이라고 부릅니다. 그러나 exact한 주소는 알려져 있지 않습니다.

가장 가까운 지구는 성수동의 "Hong-gil-dong 1-1"이라고 합니다. 하지만 이 주소도 정확하지 않습니다.

hong Gil-dong의 자취를 찾아보니, 그는 성수동에서 살았던 것으로 보입니다. 하지만 exact한 주소나 장소는 알려지지 않습니다.

hong Gil-dong은 한국의 전통적인 소설로, 많은 사람들이 그에 대한 관심을 가지고 있습니다. 그의 이야기를 통해, 많은 사람들이 한국의 전통적인 문화와 역사를 이해할 수 있습니다.

네이버나 카카오 맵 등에서 성수동을探索해 보시면, 홍gil동과 같은 다양한 장소를 찾을 수 있습니다.

inference time: 4.60010 sec 


========= llama_peft ===========
성수동의 날라다니는 홍길동의 자취는, 한국의 전통시장으로 알려진 성수동에서만 가능합니다. 홍길동은 성수동의 주변을 따라 다니며, 주변의 상점과 가게를 방문하여 소비자에게 다양한 제품과 서비스를 제공하고 있습니다. 이곳은 한국의 전통시장으로서, 많은 사람들로 인기를 얻고 있습니다.

inference time: 1.70522 sec 




'성수동의 날라다니는 홍길동의 자취는, 한국의 전통시장으로 알려진 성수동에서만 가능합니다. 홍길동은 성수동의 주변을 따라 다니며, 주변의 상점과 가게를 방문하여 소비자에게 다양한 제품과 서비스를 제공하고 있습니다. 이곳은 한국의 전통시장으로서, 많은 사람들로 인기를 얻고 있습니다.'

# Multiagent - autogen

In [8]:
from autogen import UserProxyAgent, ConversableAgent

llama_config={
    "config_list": [
        {
            "model": "meta-llama/Llama-3.2-3B-Instruct",
            "base_url": "http://localhost:8877/v1",
            "api_key": "token-abc123",
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

peft_config={
    "config_list": [
        {
            "model": "llama_peft",
            "base_url": "http://localhost:8877/v1",
            "api_key": "token-abc123",
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [9]:
# Create the agent that uses the LLM.
assistant = ConversableAgent(
    "assistant",
    llm_config=llama_config,system_message="You are an assistant who selects the most suitable agent to answer the user's question and then asks the question on their behalf."
    )

# Create the agent that represents the user in the conversation.
baemin_agent = UserProxyAgent(
    "baemin_agent",
    llm_config=peft_config,
    code_execution_config=False,system_message="당신은 배달의민족 주문관련 질문에 항상 한국어로 답변하는 agent 이다."
    )

In [14]:
assistant.initiate_chat(
    baemin_agent,
    message="안심번호와 함께 주문한 음식이 취소된 경우 어떻게 되나요?"
    )

assistant (to baemin_agent):

안심번호와 함께 주문한 음식이 취소된 경우 어떻게 되나요?

--------------------------------------------------------------------------------
Replying as baemin_agent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 04-19 08:54:58] {693} WARNING - Model llama_peft is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


baemin_agent (to assistant):

배달의민족 주문 관련 질문에 대한 답변은 다음과 같습니다. [안심번호와 함께 주문한 음식이 취소된 경우, 주문자와 주문자만이 주문 내용을 확인할 수 있습니다. 주문 내용을 확인한 후, 주문자만이 주문 내용을 취소할 수 있습니다. 주문 내용을 취소하면 주문 내용이 취소되며, 주문자만이 주문 내용을 확인할 수 있습니다.]

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 04-19 08:54:59] {693} WARNING - Model meta-llama/Llama-3.2-3B-Instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


assistant (to baemin_agent):

배달의민족의 답변을 요약하면 다음과 같습니다.

배달의민족 주문 내용을 취소할 수 있는 경우, 주문자와 주문자만이 주문 내용을 확인할 수 있습니다. 주문 내용을 확인한 후, 주문자만이 주문 내용을 취소할 수 있습니다.

--------------------------------------------------------------------------------
Replying as baemin_agent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...


BadRequestError: Error code: 400 - {'object': 'error', 'message': 'This model only supports single tool-calls at once!', 'type': 'BadRequestError', 'param': None, 'code': 400}

In [26]:
%%bash
# Add Docker's official GPG key:
sudo apt-get update
sudo apt-get install ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "${UBUNTU_CODENAME:-$VERSION_CODENAME}") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

sudo groupadd docker
sudo usermod -aG docker $USER
newgrp docker

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20240203~22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Usage: usermod [options] LOGIN

Options:
  -b, --badnames                allow bad names
  -c, --comment COMMENT         new value of the GECOS field
  -d, --home HOME_DIR           new home directory for the user account
  -e, --expiredate EXPIRE_DATE  set account expiration date to EXPIRE_DATE
  -f, --inactive INACTIVE       set password inactive after expiration
                                to INACTIVE
  -g, --gid GROUP               force use GROUP as new primary group
  -G, --groups GROUPS           new list of supplementary GROUPS
  -a, --append                  append the user to the supplemental GROUPS
 

In [15]:
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = "data"


# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    system_message="Execution of Python code and reviewing the execution results.",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [21]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llama_config,
    code_execution_config=False,  # Turn off code execution for this agent.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [22]:
chat_result = code_writer_agent.initiate_chat(
    code_writer_agent,
    #message="Write Python code to calculate the 14th Fibonacci number.",
    message="오늘 몇일이야?.",
)

code_writer_agent (to code_writer_agent):

오늘 몇일이야?.

--------------------------------------------------------------------------------
Replying as code_writer_agent. Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 04-19 08:56:32] {693} WARNING - Model meta-llama/Llama-3.2-3B-Instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


code_writer_agent (to code_writer_agent):

Today's date is not enough to solve the task. Let me try to collect more information.

Can you please provide more context or clarify what task you would like me to help you with? Are you looking to:

1. Browse the web for something specific?
2. Download or read a file?
3. Print the content of a webpage or a file?
4. Get the current date/time?
5. Check the operating system?
6. Something else?

Please provide more information, and I'll be happy to assist you.

--------------------------------------------------------------------------------
Replying as code_writer_agent. Provide feedback to code_writer_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit

>>>>>>>> TERMINATING RUN (18009ecc-e411-4452-98bc-98161d44ce93): User requested to end the conversation


In [23]:
import autogen

# Create a GroupChat with the defined agents
multi_agent_chat = autogen.GroupChat(
    agents=[assistant, baemin_agent, code_writer_agent, code_executor_agent],
    messages=[],
    max_round=5
)

# Create a GroupChatManager to manage the multi-agent conversation
manager = autogen.GroupChatManager(
    groupchat=multi_agent_chat,
    llm_config=llama_config
)

NameError: name 'code_executor_agent' is not defined

In [20]:
user_prompt = "안심번호와 함께 주문한 음식이 취소된 경우 어떻게 되나요?"
#user_prompt = "오늘 몇일이야?"

assistant.initiate_chat(manager, message=user_prompt)

NameError: name 'manager' is not defined